In [38]:
import numpy as np
import yfinance as yf
import pandas as pd
import statsmodels.api as sm
from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.graphics.tsaplots import plot_acf
import matplotlib.pyplot as plt



tickers = {
    "SPC": "^GSPC",  # Cap-weighted index
    "SPE": "RSP"     # Equal-weight ETF
}
start_date = "2005-01-01"
end_date = "2025-05-31"


In [39]:
### yahoo finance doesn't work:
spc = pd.read_csv("^GSPC_mon.csv", index_col=0, parse_dates=True)
spe = pd.read_csv("RSP_mon.csv", index_col=0, parse_dates=True)

# Combine into a single DataFrame
close_df = pd.DataFrame({
    "SPC": spc["Close"],
    "SPE": spe["Close"]
}).dropna()

# Compute returns and innovation portfolio
returns_df = close_df.pct_change().dropna()
returns_df["Innovation Portfolio"] = returns_df["SPC"] - returns_df["SPE"]
returns_df = returns_df.iloc[:-1]

In [40]:
#upload ff 3 factor data

ff_df = pd.read_csv("F-F_Research_Data_5_Factors_2x3.csv", index_col=0, skiprows=3)
ff_df = ff_df[ff_df.index.astype(str).str.match(r"^\d{6}$")]
ff_df.index = pd.to_datetime(ff_df.index.astype(str), format="%Y%m")
ff_df = ff_df.reindex(returns_df.index).dropna()

ff_df = ff_df.apply(pd.to_numeric, errors='coerce') / 100

merged_df = returns_df.join(ff_df, how='left')


excess_rets_df = pd.DataFrame()
excess_rets_df["SPC"] = merged_df["SPC"] - merged_df["RF"]
excess_rets_df["SPE"] = merged_df["SPE"] - merged_df["RF"]
excess_rets_df["Innovation Portfolio"] = merged_df["Innovation Portfolio"] - merged_df["RF"]
excess_rets_df["Mkt-RF"] = merged_df["Mkt-RF"]
excess_rets_df["SMB"] = merged_df["SMB"]
excess_rets_df["HML"] = merged_df["HML"]
excess_rets_df["RMW"] = merged_df["RMW"]
excess_rets_df["CMA"] = merged_df["CMA"]


In [41]:
#ffm = pd.read_csv(r"C:\Users\Username\OneDrive\Desktop\Invesco_project\F-F_Momentum_Factor.csv",skiprows= 13, nrows = 1180)
ffm = pd.read_csv("F-F_Momentum_Factor.csv",skiprows= 13, nrows = 1180)
ffm.rename(columns={"Unnamed: 0": "Date"}, inplace=True)

ffm["Date"] = pd.to_datetime(ffm["Date"], format="%Y%m")

ffm.set_index("Date", inplace=True)

ffm = ffm.astype(float) / 100

In [42]:
ff_df = pd.merge(ff_df, ffm, on="Date", how="inner")

In [43]:
# ff_df

In [44]:

excess_rets_df["Mom"] = ff_df["Mom"]

In [45]:
def compute_rolling_betas_and_alpha(
    excess_returns_df,
    portfolio_col,
    window=36,
    beta_MKT=False,
    beta_SMB=True,
    beta_HML=True,
    beta_RMW=True,
    beta_CMA=True,
    beta_Momentum=False
):
    factor_flags = {
        "Mkt-RF": beta_MKT,
        "SMB": beta_SMB,
        "HML": beta_HML,
        "RMW": beta_RMW,
        "CMA": beta_CMA,
        "Mom": beta_Momentum
    }

    selected_factors = [factor for factor, include in factor_flags.items() if include]

    results = {
        "alpha": [],
        "r_squared": [],
        "alpha_tstat": [],
        "alpha_pval": [],
        "sortino": []
    }

    for factor in selected_factors:
        results[f"beta_{factor}"] = []

    index = []

    for i in range(window, len(excess_returns_df)):
        window_df = excess_returns_df.iloc[i - window:i]

        y = window_df[portfolio_col]
        X = window_df[selected_factors]
        X = sm.add_constant(X)

        model = sm.OLS(y, X).fit()

        results["alpha"].append(model.params.get("const", float("nan")))
        results["r_squared"].append(model.rsquared)
        results["alpha_tstat"].append(model.tvalues.get("const", float("nan")))
        results["alpha_pval"].append(model.pvalues.get("const", float("nan")))

        downside_std = y[y < 0].std()
        sortino = y.mean() / downside_std if downside_std != 0 else float("nan")
        results["sortino"].append(sortino)

        for factor in selected_factors:
            results[f"beta_{factor}"].append(model.params.get(factor, float("nan")))

        index.append(excess_returns_df.index[i])

    return pd.DataFrame(results, index=index)


In [46]:
import ipywidgets as widgets
from ipywidgets import interact
import matplotlib.pyplot as plt

In [47]:
# import ipywidgets as widgets
# from ipywidgets import interact
# import matplotlib.pyplot as plt
# from IPython.display import display
# from IPython.display import HTML
#
# def plot_rolling(window=12,
#                  factors=["Mkt-RF", "SMB", "HML", "RMW", "CMA", "Mom"],
#                  table_metrics=["r_squared", "alpha_tstat", "alpha_pval", "sortino"]):
#
#     selected_df = compute_rolling_betas_and_alpha(
#         excess_rets_df,
#         portfolio_col="Innovation Portfolio",
#         window=window,
#         beta_MKT="Mkt-RF" in factors,
#         beta_SMB="SMB" in factors,
#         beta_HML="HML" in factors,
#         beta_RMW="RMW" in factors,
#         beta_CMA="CMA" in factors,
#         beta_Momentum="Mom" in factors
#     )
#
#     # ---------- Plot alpha and betas ----------
#     plot_cols = ["alpha"] + [f"beta_{f}" for f in factors if f"beta_{f}" in selected_df.columns]
#     selected_df[plot_cols].plot(figsize=(12, 6), title=f"{window}-Month Rolling Alpha & Betas")
#     plt.grid(True)
#     plt.tight_layout()
#     plt.show()
#
#     # ---------- Summary table ----------
#     # Compute all available metrics
#     all_metrics = {
#         "Mean": selected_df.mean(),
#         "Std Dev": selected_df.std(),
#         "r_squared": selected_df["r_squared"].mean() if "r_squared" in selected_df.columns else None,
#         "alpha_tstat": selected_df["alpha_tstat"].mean() if "alpha_tstat" in selected_df.columns else None,
#         "alpha_pval": selected_df["alpha_pval"].mean() if "alpha_pval" in selected_df.columns else None,
#         "sortino": selected_df["sortino"].mean() if "sortino" in selected_df.columns else None
#     }
#
#     # Filter metrics based on user selection
#     selected_metrics = {k: v for k, v in all_metrics.items() if k in table_metrics and v is not None}
#
#     # Build summary DataFrame for selected rows and columns
#     row_names = ["alpha"] + [f"beta_{f}" for f in factors if f"beta_{f}" in selected_df.columns]
#     summary = pd.DataFrame({metric: selected_df[row_names].mean() if metric == "Mean"
#                             else selected_df[row_names].std() if metric == "Std Dev"
#                             else [selected_metrics[metric]] * len(row_names)
#                             for metric in selected_metrics})
#
#     summary.index = row_names
#     summary = summary.round(4)
#
#     # Store globally
#     global latest_summary_table
#     latest_summary_table = summary
#
#     display(HTML("<h4>Average Results from Rolling Factor Mode</h4>"))
#     display(summary)
#
#
# interact(
#     plot_rolling,
#     window=widgets.IntSlider(min=6, max=60, step=6, value=12),
#     factors=widgets.SelectMultiple(
#         options=["Mkt-RF", "SMB", "HML", "RMW", "CMA", "Mom"],
#         value=("SMB", "HML", "RMW", "CMA", "Mom"),
#         description='Factors',
#         style={'description_width': 'initial'}
#     ),
#     table_metrics=widgets.SelectMultiple(
#         options=["Mean", "Std Dev", "r_squared", "alpha_tstat", "alpha_pval", "sortino"],
#         value=("Mean", "Std Dev"),
#         description='Summary Columns',
#         style={'description_width': 'initial'}
#     )
#
# )
#


In [48]:
from ipywidgets import (
    interactive_output, IntSlider, VBox, HBox, Checkbox, Dropdown, Label, Button
)
from IPython.display import display, HTML

# Define options
factor_options = ["Mkt-RF", "SMB", "HML", "RMW", "CMA", "Mom"]
metric_options = ["Mean", "Std Dev", "r_squared", "alpha_tstat", "alpha_pval", "sortino"]
y_options = ["SPC", "SPE", "Innovation Portfolio"]

# Helper functions
def create_checkboxes(options, default=[]):
    return [Checkbox(value=(opt in default), description=opt, indent=False) for opt in options]

def get_selected_options(checkboxes):
    return [cb.description for cb in checkboxes if cb.value]

# Create widgets
factor_checkboxes = create_checkboxes(factor_options, default=["SMB", "HML", "RMW", "CMA", "Mom"])
metric_checkboxes = create_checkboxes(metric_options, default=["Mean", "Std Dev"])
y_selector = Dropdown(options=y_options, value="Innovation Portfolio", description="Portfolio")
window_slider = IntSlider(min=6, max=60, step=6, value=12, description="Window")
run_button = Button(description="Run", button_style='success')

# Store output to update
output_area = widgets.Output()

# Define wrapped function for plotting
def wrapped_plot_rolling(button=None):  # optional param for button click
    with output_area:
        output_area.clear_output()
        factors = get_selected_options(factor_checkboxes)
        table_metrics = get_selected_options(metric_checkboxes)
        y_var = y_selector.value
        window = window_slider.value

        selected_df = compute_rolling_betas_and_alpha(
            excess_rets_df,
            portfolio_col=y_var,
            window=window,
            beta_MKT="Mkt-RF" in factors,
            beta_SMB="SMB" in factors,
            beta_HML="HML" in factors,
            beta_RMW="RMW" in factors,
            beta_CMA="CMA" in factors,
            beta_Momentum="Mom" in factors
        )

        # Plot rolling alpha and betas
        plot_cols = ["alpha"] + [f"beta_{f}" for f in factors if f"beta_{f}" in selected_df.columns]
        selected_df[plot_cols].plot(figsize=(12, 6), title=f"{window}-Month Rolling Alpha & Betas")
        plt.grid(True)
        plt.tight_layout()
        plt.show()

        # Summary table
        all_metrics = {
            "Mean": selected_df.mean(),
            "Std Dev": selected_df.std(),
            "r_squared": selected_df["r_squared"].mean() if "r_squared" in selected_df.columns else None,
            "alpha_tstat": selected_df["alpha_tstat"].mean() if "alpha_tstat" in selected_df.columns else None,
            "alpha_pval": selected_df["alpha_pval"].mean() if "alpha_pval" in selected_df.columns else None,
            "sortino": selected_df["sortino"].mean() if "sortino" in selected_df.columns else None
        }

        selected_metrics = {k: v for k, v in all_metrics.items() if k in table_metrics and v is not None}
        row_names = ["alpha"] + [f"beta_{f}" for f in factors if f"beta_{f}" in selected_df.columns]

        summary = pd.DataFrame({metric: selected_df[row_names].mean() if metric == "Mean"
                                else selected_df[row_names].std() if metric == "Std Dev"
                                else [selected_metrics[metric]] * len(row_names)
                                for metric in selected_metrics})

        summary.index = row_names
        summary = summary.round(4)

        global latest_summary_table
        latest_summary_table = summary

        display(HTML("<h4>Average Results from Rolling Factor Model</h4>"))
        display(summary)

# Bind the run button to the function
run_button.on_click(wrapped_plot_rolling)

# Display full UI
ui = VBox([
    HBox([Label("Factors:"), VBox(factor_checkboxes)]),
    HBox([Label("Summary Columns:"), VBox(metric_checkboxes)]),
    y_selector,
    window_slider,
    run_button,
    output_area
])

display(ui)